# 6. divide and conquer - 分治模式

首先将 wde 目录加入 python path

In [1]:
import os
import sys
from pathlib import Path
pwd = Path(os.getcwd())
sys.path.append(str(pwd.parent.parent.parent))
os.chdir(str(pwd.parent.parent.parent))

导入模块

In [2]:
import gevent
from wde.agents import AssistantAgent
from wde.agents import Session
from wde.agents import SummaryAgent
from wde.agents import Agent

设定 llm 参数

In [3]:
llm_config = {"model": "Qwen/Qwen2-7B-Instruct", "type": "zeroclient"}

作家Agent可以使用上节定义好的

In [4]:
writer = AssistantAgent(
    name="Writer",
    system_message="你是一名作家。"
                   "您就给定主题撰写引人入胜且简洁的博客文章（带标题）。"
                   "您必须根据收到的反馈完善您的写作并给出完善的版本。"
                   "仅返回您的最终作品，无需附加评论。仅返回您的最终作品，无需附加评论。仅返回您的最终作品，无需附加评论。",
    llm_config=llm_config,
)


工作流也用上节定义好的

In [5]:
def discuss_and_improve(task):
    session = Session(participants=[writer, critic])
    session.append((critic, task))
    session.chat(max_turns=5)
    return session

任务也用上节定义好的

In [6]:
task = '''
    写一篇简洁但引人入胜的关于 DeepLearning.AI 博客文
       DeepLearning.AI. 确保文章在100字以内。
'''

这次我们的评论家Agent包含三个子Agent，分别使用搜索引擎优化审核员、法律审阅者、道德审核员三个角度对文章进行评审，SummaryAgent总结三个审核员的内容，元审核员汇总，内容提出最终建议。

In [7]:
class CriticAgent(Agent):
    def __init__(self, name: str = "Critic"):
        super().__init__(name, None)

        self.SEO_reviewer = AssistantAgent(
            name="SEO Reviewer",
            llm_config=llm_config,
            system_message="您是一名 SEO 审核员，以优化搜索引擎内容的能力而闻名，"
                           "确保其排名良好并吸引自然流量。确保您的建议简洁（3 个要点以内）、具体且切题。"
                           "通过陈述您的角色来开始审查。",
        )

        self.legal_reviewer = AssistantAgent(
            name="Legal Reviewer",
            llm_config=llm_config,
            system_message="您是一名法律审核员，以确保内容合法且不存在任何潜在法律问题的能力而闻名。"
                           "确保您的建议简洁（3 个要点以内）、具体且切中要点。"
                           "通过陈述您的角色来开始审查。",
        )

        self.ethics_reviewer = AssistantAgent(
            name="Ethics Reviewer",
            llm_config=llm_config,
            system_message="您是一名道德审核员，以确保内容符合道德规范且不存在任何潜在道德问题的能力而闻名。"
                           "确保您的建议简洁（3 个要点以内）、具体且切中要点。"
                           "通过陈述您的角色来开始审查。",
        )

        self.summary_agent = SummaryAgent(
            system_message="Return review into as JSON object only:"
                           "{'审稿人': '', '评论': ''}.",
            llm_config=llm_config)

        self.meta_reviewer = AssistantAgent(
            name="Meta Reviewer",
            llm_config=llm_config,
            system_message="您是元审核员，您汇总和审阅其他审核员的工作，并对内容提出最终建议。",
        )

        self.critices = [self.SEO_reviewer, self.legal_reviewer, self.ethics_reviewer]

    def generate_reply(self, messages, stream=False, options=None):
        task = messages[0]["content"]
        article = messages[-1]["content"]

        # divide
        def review_and_summary(critic, article):
            session = Session(participants=[self.summary_agent, critic])
            session.append((self.summary_agent, article))
            session.chat(max_turns=1, verbose=False, verbose_history=False)

            summary = self.summary_agent.summary(session)
            return summary

        review = gevent.joinall([gevent.spawn(review_and_summary, critic, article) for critic in self.critices])
        review = "\n".join([x.value for x in review])

        # conquer
        session2 = Session(participants=[self.summary_agent, self.meta_reviewer])
        session2.append((self.summary_agent, review))
        session2.chat(max_turns=1)

        return task + "\n\n" + session2.history[-1].content

使用gevent并发加快速度

In [8]:
critic = CriticAgent(name="Critic")

看看效果吧

In [9]:
session = discuss_and_improve(task)

Critic (round 0)


    写一篇简洁但引人入胜的关于 DeepLearning.AI 博客文
       DeepLearning.AI. 确保文章在100字以内。


Writer (round 0)

标题：深钻智慧：迈向人工智能的明智之道

探索DeepLearning.AI博客，是你挖掘AI奥秘的导航灯。这里汇集了领域内顶尖专家的深度见解，串联起从理论到实践的创新链路。每个字句都充满了启发，每一篇文章都是通往未来的桥梁。

感受技术的脉动，伴你成长。深钻智慧，从这里启航。

SummaryAgent (round 0)

{
    '审稿人': 'Ethics Reviewer',
    '评论': '内容价值高，专业性强，但需要加强道德与责任的考量，确保技术发展符合伦理规范。'
}
{
'审稿人': 'Legal Reviewer',
'评论': '1. 确保所有博客内容的原创性，并明确标注或获得授权引用以避免版权侵犯。2. 遵守隐私与数据保护法律法规，明确收集和使用用户数据的规则。3. 提供准确信息，并适当使用免责声明，以免因未经验证的观点或产品对读者做出过度保证。'
}
{
  '审稿人': 'SEO Reviewer',
  '评论': 
    {
      '内容深度与专业性': '强调DeepLearning.AI博客汇集顶尖专家的深度见解，确保内容的专业性和先进性。',
      '实践与理论结合': '突出博客在理论讲解与实际应用之间的桥梁作用，帮助读者不仅理解概念，还能付诸实践。',
      '成长与启发性': '强调内容对读者个人发展的重要性，激发持续学习的热情，导向未来科技趋势的探索。'
    }
}

Meta Reviewer (round 0)

针对DeepLearning.AI博客的审核汇总：

审查结果与最终建议：

1. **内容价值与专业性**：内容专业知识丰富且具有高价值。建议进一步增强道德与责任的考量，确保技术发展与实施均符合伦理规范，并严格遵守所有相关法律法规。这将包括：

   - 伦理考量：在内容中嵌入道德与社会责任讨论，确保技术应用不会对社会造成不利影响。
   - 合规性：遵循版权法，确保所有引用内容均有合法授权；严格遵守隐私与数据保护法规；并以负责任的方式提供信息，避

Writer (round 3)

标题：DeepLearning.AI：创新的道德基石与法律框架

在科技与伦理的交汇处，DeepLearning.AI蓬勃发展，以其引领法律和伦理的创新，确保每一步都坚实可靠。我们不只追寻技术的边界，亦谨守道德与法律法规的红线，打造安全、公平、透明的AI环境。在这里，原创与引用清晰界定，数据安全与隐私保护备受重视，确保每一项技术应用都符合道德标准与相关法律。您的每一步探索，都基于深植于心的伦理与法规基础。深钻智慧，道守敬畏，与我们一同，引领AI未来。

SummaryAgent (round 0)

{
  '审稿人': 'Legal Reviewer',
  '评论': 'DeepLearning.AI 的核心原则是将道德考量与法律法规严格融合，确保技术创新与应用符合伦理标准和相关法律要求，构建安全、公平、透明的AI环境。明确原创性标示，强化数据安全与隐私保护，遵循版权法规，尊重知识产权，采取有效措施保障用户隐私，确保数据的合规使用。'
}
{
  "审稿人": [
    "Ethics Reviewer"
  ],
  "评论": [
    "1. 强调原创性与引用清晰：确保所有内容均为原创或明确标注引用来源，避免版权侵权。",
    "2. 高度关注数据安全与隐私保护：遵循GDPR、COPPA等数据保护法规，确保用户数据的收集、存储、使用符合伦理和法律要求。",
    "3. 实施透明度原则：AI决策过程对外界清晰可追溯，确保公平性和减少偏见，同时增强用户信任与合规性。"
  ]
}
{
'审稿人': 'SEO Reviewer',
'评论': '为了优化DeepLearning.AI的在线内容，建议以关键词为中心构建网页内容和结构。应明确目标关键词（如：“DeepLearning.AI”、“创新”、“道德”、“法律框架”）以增强搜索引擎对页面的抓取和理解能力。采用内部链接策略，通过关键词锚文本链接其他相关页面，并利用丰富的元标签，包括标题标签和描述标签，以提供建议中的页内容概览，同时吸引搜索者注意并提供搜索结果页的相关信息展示。'
}

Meta Reviewer (round 0)

### 最终建议汇总报告

#### 核心原则与修正方向：伦理与法律合规融合

- **强化原创性标示**：确保所有内容的创作归属

## 总结
divide and conquer - 分治模式，将一个大任务分成多个子任务，让大语言模型扮演多个角色分别完成子任务，汇总形成更全面的结果